In [1]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from main import train_model
from classes.model import Model
from classes.data_handler import DataHandler, Filter
from classes.results_handler import ResultsHandler

### **PHASE 1**

#### 1. **Charger les données pour Lille** depuis le fichier `data/ValeursFoncieres-2022.csv`
#### 2. **Filtrer les biens de 4 pièces uniquement** : `Nombre pieces principales == 4`
#### 3. **Créer deux jeux de données distincts** :
    - Un jeu avec uniquement les **appartements**
    - Un jeu avec uniquement les **maisons**
#### 4. **Pour chaque jeu, ne conservez que les colonnes suivantes** :
    - `Surface reelle bati`
    - `Nombre pieces principales`
    - `Type local`
    - `Surface terrain` (si disponible)
    - `Nombre de lots`
    - `Valeur fonciere` (pour calculer le `prix_m2`)

In [2]:
filters_apartments = [
    Filter('Commune', '==', "lille"),
    Filter('Type local', '==', "appartement"),
    Filter('Nature mutation', '==', 'vente'),
    Filter('Valeur fonciere', 'notnull', None),
    Filter('Surface reelle bati', 'notnull', None),
    Filter('Nombre pieces principales', '==', 4),
]

filters_houses = [
    Filter('Commune', '==', "lille"),
    Filter('Type local', '==', "maison"),
    Filter('Nature mutation', '==', 'vente'),
    Filter('Valeur fonciere', 'notnull', None),
    Filter('Surface reelle bati', 'notnull', None),
    Filter('Nombre pieces principales', '==', 4),
]

df_apartments = DataHandler.extract_data("data/ValeursFoncieres-2022.txt", filters_apartments)
df_apartments = df_apartments[["Surface reelle bati", "Nombre pieces principales", "Type local", "Nombre de lots", "Valeur fonciere"]]
df_houses = DataHandler.extract_data("data/ValeursFoncieres-2022.txt", filters_houses)
df_houses = df_houses[["Surface reelle bati", "Nombre pieces principales", "Type local", "Surface terrain", "Nombre de lots", "Valeur fonciere"]]

df_houses.head()

Data extracted
Data extracted


,Surface reelle bati,Nombre pieces principales,Type local,Surface terrain,Nombre de lots,Valeur fonciere
2685806,165.0,4.0,Maison,121.0,0,"305000,00"
2685934,64.0,4.0,Maison,127.0,0,"226900,00"
2686633,70.0,4.0,Maison,192.0,0,"205890,00"
2686700,62.0,4.0,Maison,96.0,0,"159500,00"
2686810,72.0,4.0,Maison,135.0,0,"259900,00"


#### 5. **Créer la variable cible** :
    
    ```python
    prix_m2 = Valeur fonciere / Surface reelle bati
    ```    
#### 6. **Nettoyer les données** :
    - Supprimer les lignes avec valeurs manquantes sur les colonnes utilisées
    - Identifier et retirer les valeurs aberrantes (prix au m² trop faible ou trop élevé)

In [3]:
df_apartments = DataHandler.add_data(df_apartments)
df_apartments = DataHandler.clean_data(df_apartments)

df_houses = DataHandler.add_data(df_houses)
df_houses = DataHandler.clean_data(df_houses)

df_houses.head()

Data converted
Dataset upgraded!
Data cleaned
Data converted
Dataset upgraded!
Data cleaned


,Surface reelle bati,Nombre pieces principales,Type local,Surface terrain,Nombre de lots,Valeur fonciere,prix_m2
2685806,165.0,4.0,Maison,121.0,0,"305000,00",1848.484848
2685934,64.0,4.0,Maison,127.0,0,"226900,00",3545.312500
2686633,70.0,4.0,Maison,192.0,0,"205890,00",2941.285714
2686700,62.0,4.0,Maison,96.0,0,"159500,00",2572.580645
2686810,72.0,4.0,Maison,135.0,0,"259900,00",3609.722222


#### **7. Préparer les données pour l'entraînement**

- Variables explicatives : `X`
- Variable cible : `y = prix_m2`
- Division en jeu d'entraînement (80%) et test (20%) avec `train_test_split`

In [4]:
model_apartments = Model(df_apartments)
model_houses = Model(df_houses)

model_apartments.clean_outliers("prix_m2")
model_houses.clean_outliers("prix_m2")

model_apartments.set_data(["Surface reelle bati", "Nombre pieces principales", "Nombre de lots"])
model_houses.set_data(["Surface reelle bati", "Surface terrain", "Nombre pieces principales", "Nombre de lots"])

#### **8. Entraîner les modèles de base avec `scikit-learn`**

- `LinearRegression`
- `DecisionTreeRegressor`
- `RandomForestRegressor`

In [5]:
model_apartments_linear = model_apartments.train_model(LinearRegression)
model_apartments_decision_tree = model_apartments.train_model(DecisionTreeRegressor)
model_apartments_rendom_forest = model_apartments.train_model(RandomForestRegressor)

model_houses_linear = model_houses.train_model(LinearRegression)
model_houses_decision_tree = model_houses.train_model(DecisionTreeRegressor)
model_houses_random_forest = model_houses.train_model(RandomForestRegressor)

#### **9. Optimiser les modèles d’arbres avec `GridSearchCV`**

- Appliquer une recherche d’hyperparamètres sur les arbres pour améliorer les résultats

#### **10. Ajouter un modèle moderne : `XGBRegressor`**

In [6]:
evaluation_results = {}

decision_tree_optimized_results = train_model(
    ["lille"],
    ["maison", "appartement"],
    DecisionTreeRegressor,
    {
        "splitter": ["best", "random"],
        "max_depth": [None, 1, 2],
        "min_samples_split": [2, 3],
        "min_samples_leaf": [1, 2],
        "min_weight_fraction_leaf": [0, 0.5],
        "max_features": [None, 2, 5]
    }
)

evaluation_results['DecisionTreeOptimized'] = decision_tree_optimized_results['results']

Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best parameters: {'max_depth': 1, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'min_weight_fraction_leaf': 0, 'splitter': 'random'}
Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best parameters: {'max_depth': 2, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0, 'splitter': 'random'}


In [7]:
random_forest_optimized_results = train_model(
    ["lille"],
    ["maison", "appartement"],
    RandomForestRegressor,
    {
        "n_estimators": [100, 75, 125],
        "max_depth": [None, 1, 2],
        "min_samples_split": [2, 3],
        "min_samples_leaf": [1, 2],
        "min_weight_fraction_leaf": [0, 0.5],
        "max_features": [1.0, 2, 5]
    }
)

evaluation_results['RandomForestOptimized'] = random_forest_optimized_results['results']

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters: {'max_depth': 1, 'max_features': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.5, 'n_estimators': 100}
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters: {'max_depth': 1, 'max_features': 1.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0, 'n_estimators': 100}


In [8]:
xgboost_optimized_results = train_model(
    ["lille"],
    ["maison", "appartement"],
    XGBRegressor,
    {
        'learning_rate': [0.1, 0.2],
        'gamma': [1, 2],
        'max_depth': [6, 7],
        'min_child_weight': [5, 6],
        'max_delta_step': [0, 1],
        'subsample': [0.5, 1],
        'sampling_method': ['uniform'],
        'colsample_bytree': [1, 0.5],
        'colsample_bylevel': [1, 0.5],
        'colsample_bynode': [1, 0.5],
        'reg_lambda': [1, 2],
        'reg_alpha': [0.5, 1],
        'n_estimators': [10, 20]
    }
)

evaluation_results['XGBoostOptimized'] = xgboost_optimized_results['results']

Fitting 5 folds for each of 4096 candidates, totalling 20480 fits
Best parameters: {'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 1, 'learning_rate': 0.1, 'max_delta_step': 1, 'max_depth': 7, 'min_child_weight': 5, 'n_estimators': 10, 'reg_alpha': 1, 'reg_lambda': 1, 'sampling_method': 'uniform', 'subsample': 0.5}
Fitting 5 folds for each of 4096 candidates, totalling 20480 fits
Best parameters: {'colsample_bylevel': 0.5, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 2, 'learning_rate': 0.2, 'max_delta_step': 0, 'max_depth': 6, 'min_child_weight': 6, 'n_estimators': 20, 'reg_alpha': 1, 'reg_lambda': 2, 'sampling_method': 'uniform', 'subsample': 0.5}


#### **11. Évaluer les performances de tous les modèles**

- Utiliser la métrique **MSE** (`mean_squared_error`)
- Comparer les performances de tous les modèles testés
- Afficher un **tableau comparatif clair** pour :
    - les **appartements**
    - les **maisons**

In [9]:
ResultsHandler.show_metrics_comparison(evaluation_results)

model_name: DecisionTreeOptimized
model_name: RandomForestOptimized
model_name: XGBoostOptimized
                Model  City        Type  R² Train  R² Test  MSE Train  MSE Test  RMSE Train  RMSE Test  MAE Train  MAE Test
DecisionTreeOptimized Lille      maison    0.0162  -0.0433     0.9838    0.9416      0.9919     0.9703     0.7913    0.7204
DecisionTreeOptimized Lille appartement    0.0129  -0.0118     0.9871    1.0231      0.9935     1.0115     0.7950    0.8336
RandomForestOptimized Lille      maison    0.0281  -0.0508     0.9719    0.9483      0.9859     0.9738     0.7854    0.7188
RandomForestOptimized Lille appartement    0.0781   0.0944     0.9219    0.9158      0.9601     0.9570     0.7680    0.7852
     XGBoostOptimized Lille      maison    0.1526  -0.0406     0.8474    0.9391      0.9206     0.9691     0.7142    0.7127
     XGBoostOptimized Lille appartement    0.0654   0.0756     0.9346    0.9347      0.9667     0.9668     0.7773    0.8013


Le modèle retenu pour les appartements selon les résultats MSE est le RandomForest
Le modèle retenu pour les maisons selon les résultats MSE est le XGBoost

### **Phase 2 : Test de généralisation sur Bordeaux**

#### 1. **Charger les données de Bordeaux** depuis le fichier `data/bordeaux_2022.csv`.
#### 2. **Appliquer exactement le même filtrage que pour Lille.**
#### 3. **Séparer les logements en deux catégories** :
    - **Appartements**
    - **Maisons**
#### 4. **Pour chaque catégorie, effectuer les mêmes préparations que dans la phase 1** :
    - Calculer `prix_m2 = Valeur fonciere / Surface reelle bati`
    - Conserver uniquement les colonnes suivantes :
        - `Surface reelle bati`
        - `Nombre pieces principales`
        - `Type local`
        - `Surface terrain`
        - `Nombre de lots`
    - Nettoyage des données (valeurs manquantes, outliers)

In [ ]:
filters_apartments = [
    Filter('Commune', '==', "bordeaux"),
    Filter('Type local', '==', "appartement"),
    Filter('Nature mutation', '==', 'vente'),
    Filter('Valeur fonciere', 'notnull', None),
    Filter('Surface reelle bati', 'notnull', None),
    Filter('Nombre pieces principales', '==', 4),
]

filters_houses = [
    Filter('Commune', '==', "bordeaux"),
    Filter('Type local', '==', "maison"),
    Filter('Nature mutation', '==', 'vente'),
    Filter('Valeur fonciere', 'notnull', None),
    Filter('Surface reelle bati', 'notnull', None),
    Filter('Nombre pieces principales', '==', 4),
]

df_bordeaux_apartments = DataHandler.extract_data("data/ValeursFoncieres-2022.txt", filters_apartments)
df_bordeaux_apartments = df_bordeaux_apartments[["Surface reelle bati", "Nombre pieces principales", "Type local", "Nombre de lots", "Valeur fonciere"]]
df_bordeaux_houses = DataHandler.extract_data("data/ValeursFoncieres-2022.txt", filters_houses)
df_bordeaux_houses = df_bordeaux_houses[["Surface reelle bati", "Nombre pieces principales", "Type local", "Surface terrain", "Nombre de lots", "Valeur fonciere"]]

df_bordeaux_apartments = DataHandler.add_data(df_bordeaux_apartments)
df_bordeaux_apartments = DataHandler.clean_data(df_bordeaux_apartments)

df_bordeaux_houses = DataHandler.add_data(df_bordeaux_houses)
df_bordeaux_houses = DataHandler.clean_data(df_bordeaux_houses)

df_bordeaux_houses.head()

#### 5. **Réutiliser les modèles entraînés sur Lille (phase 1)** :
    - **Ne pas réentraîner les modèles**
    - Appliquer directement les modèles (un pour les appartements, un pour les maisons)
    - Prédire les `prix_m2` sur les données de Bordeaux
    - Calculer la **MSE** pour chaque prédiction
#### 6. **Comparer les performances entre Lille et Bordeaux pour chaque type de logement** :
    - Le modèle est-il aussi performant sur Bordeaux ?
    - Quels écarts de performance observez-vous ?
    - Quels facteurs peuvent expliquer ces différences ?
    - Le modèle généralise-t-il mieux sur un type de bien que sur l'autre ?

In [ ]:
evaluation_results = {}

xgboost_optimized_results = train_model(
    ["lille"],
    ["maison"],
    XGBRegressor,
    {
        'learning_rate': [0.1],
        'gamma': [1],
        'max_depth': [7],
        'min_child_weight': [5],
        'max_delta_step': [1],
        'subsample': [0.5],
        'sampling_method': ['uniform'],
        'colsample_bytree': [1],
        'colsample_bylevel': [1],
        'colsample_bynode': [1],
        'reg_lambda': [1],
        'reg_alpha': [1],
        'n_estimators': [10]
    },
    comparison_city="bordeaux"
)

evaluation_results['XGBoostOptimized'] = xgboost_optimized_results['results']

random_forest_optimized_results = train_model(
    ["lille"],
    ["appartement"],
    RandomForestRegressor,
    {
        "n_estimators": [100],
        "max_depth": [1],
        "min_samples_split": [2],
        "min_samples_leaf": [2],
        "min_weight_fraction_leaf": [0],
        "max_features": [1.0]
    },
    comparison_city="bordeaux"
)

evaluation_results['RandomForestOptimized'] = random_forest_optimized_results['results']

ResultsHandler.show_metrics_comparison(evaluation_results)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters: {'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 1, 'learning_rate': 0.1, 'max_delta_step': 1, 'max_depth': 7, 'min_child_weight': 5, 'n_estimators': 10, 'reg_alpha': 1, 'reg_lambda': 1, 'sampling_method': 'uniform', 'subsample': 0.5}
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters: {'max_depth': 1, 'max_features': 1.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0, 'n_estimators': 100}
model_name: XGBoostOptimized
model_name: RandomForestOptimized
                Model  City        Type  R² Train  R² Test  R² Comparison  MSE Train  MSE Test  MSE Comparison  RMSE Train  RMSE Test  RMSE Comparison  MAE Train  MAE Test  MAE Comparison
     XGBoostOptimized Lille      maison    0.1526  -0.0406        -3.8545     0.8474    0.9391         11.0641      0.9206     0.9691           3.3263     0.7142    0.7127          2.9950
Random

Si on compare le MSE entre Lille et Bordeaux, on passe de 0.93 à 11.06 sur les maisons et 0.91 à 2.29 sur les appartements. Le modèle sous performe largement sur Bordeaux.
Les prix de l'immobilier sont extrêmement lié à la localisation et prédire les prix d'une ville cotiêre et bourgeoise avec un outil entrainé sur une ville populaire produit forcément de grands écarts, surtout en ce qui concerne les maisons. Ces écarts sont moins flagrants en ce qui concerne les appartements, peut être parce que ceux-ci ne sont pas aussi prisés par la population aisée et en tant que résidence secondaire.